In [79]:
import pandas as pd
import json

import time
from datetime import datetime

In [63]:
path = 'C:/Users/Martin/Desktop/data_python/data_python_conda/'

with open(path+'timeseries.json') as json_data:
    data = json.load(json_data)

timeseries = pd.DataFrame.from_dict(data, orient='index').T

In [92]:
# Vytvori dictionari vsech akcii a jejich mesicni adjusted ceny
below_cat = pd.read_json(path+'monthly_adjusted_stock_timeseries_below-CAT.json').T
above_cat = pd.read_json(path+'monthly_adjusted_stock_timeseries_above-CAT.json').T

all_stocks = pd.concat([below_cat, above_cat])
all_stocks = all_stocks.sort_index().T

In [88]:
# Zmerguje timeseries s cenami akcii
for year in timeseries:
    for company in timeseries[year]:
        
        try:
            temp = {}
            for date, information in all_stocks[company]['Monthly Adjusted Time Series'].items():
                date = datetime.strptime(date, '%Y-%m-%d')
                if date.year != int(year):
                    continue
                elif date.year < int(year):
                    break
                temp[date] = information['5. adjusted close']
            timeseries[year][company] = temp
        except KeyError as e:
            pass
            


In [91]:
# Ted mame timeseries se vsemi akciemi sp500 za dany rok a mesicnimi adjusted cenami za akcii
timeseries['2020']['AAPL']


# A dal sme v haji, protoze ted bychom meli spocitat market cap, problem ale je ze k tomu potrebujeme shares outstanding,
# ktere se casem meni... Da se to zjistit z https://www.alphavantage.co/documentation/#balance-sheet ale jenom 5 let nazpet

{datetime.datetime(2020, 12, 21, 0, 0): '128.2300',
 datetime.datetime(2020, 11, 30, 0, 0): '119.0500',
 datetime.datetime(2020, 10, 30, 0, 0): '108.6723',
 datetime.datetime(2020, 9, 30, 0, 0): '115.6103',
 datetime.datetime(2020, 8, 31, 0, 0): '128.8175',
 datetime.datetime(2020, 7, 31, 0, 0): '105.8814',
 datetime.datetime(2020, 6, 30, 0, 0): '90.8751',
 datetime.datetime(2020, 5, 29, 0, 0): '79.2018',
 datetime.datetime(2020, 4, 30, 0, 0): '72.9953',
 datetime.datetime(2020, 3, 31, 0, 0): '63.1790',
 datetime.datetime(2020, 2, 28, 0, 0): '67.9170',
 datetime.datetime(2020, 1, 31, 0, 0): '76.7139'}